PARTE 1


Primero vamos a comenzar descargando 100 videos por categoria (coches, mascotas y comida)

In [2]:
import yt_dlp
import os
import random
import time

def descargar_videos(consulta, directorio_destino, cantidad=100):
    """
    Busca y descarga videos de Youtube.
    
    Args:
        consulta: La consulta de busqueda
        directorio_destino: El directorio donde se guardaran los videos
        cantidad: La cantidad de videos a descargar.
    """
    ydl_opts = {
        'outtmpl': directorio_destino + '/%(title)s-%(id)s.%(ext)s',
        'format': 'bestvideo[height<=240][ext=mp4]/best[ext=mp4]/best', # Formato de video a descargar
        'ignoreerrors': True,
        'quiet': True, #Modo silencioso para evitar mucha salida en consola
        'nocheckcertificate': True, # Para evitar errores de certificado
        'ratelimit': 1024*1024, # limita la velocidad de descarga a 1MB/s para no saturar la red
        'throttling': 0.5, # Introducimos retardo entre descargas para evitar bloqueos
        'no-check-formats': True, # Evita que se compruebe el formato de los videos
        'merge_output_format': 'mp4', # Formato de salida para todos los videos
        'max_downloads': cantidad,
        }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([f"ytsearch{cantidad}:{consulta}"])
            print(f"Descarga de '{consulta}' completada. {cantidad} videos guardados en {directorio_destino}")
    except yt_dlp.utils.DownloadError as e:
        print(f"Error en la busqueda o descarga de '{consulta}': {e}")
    except Exception as e:
        print(f"Error al descargar videos: '{consulta}': {e}")

def descargar_categorias(categorias, directorio_base="videos", cantidad=100):
    """
    Descarga todos los videos de las categorias especificadas.
    
    Args:
        categorias: Lista de categorias a descargar.
        directorio_base: Directorio base donde se guardarán los videos.
        cantidad: Cantidad de videos a descargar.
    """
    for categoria, consulta in categorias.items():
        directorio_categoria = os.path.join(directorio_base, categoria)
        os.makedirs(directorio_categoria, exist_ok=True) # Crea el directorio si no existe
        descargar_videos(consulta, directorio_categoria, cantidad)
        time.sleep(random.randint(5, 15)) # Retardo aleatorio para evitar bloqueos

# Definir las categorias y consultas a buscar
categorias = {
    "coches": "coches deportivos",
    "mascotas": "perros graciosos",
    "comida": "recetas faciles",
}

# Descargar los videos
descargar_categorias(categorias, cantidad=100)
        

ERROR: m3u8 download detected but ffmpeg could not be found. Please install
ERROR: Unable to download video: [WinError 2] El sistema no puede encontrar el archivo especificado: 'videos\\coches\\Mecum Kissimmee： Saturday, January 18, 2025 2025-01-18 22_11-1tSIO9uw4r8.mp4.part'


ERROR: unable to download video data: HTTP Error 416: Requested range not satisfiable


Error al descargar videos: 'coches deportivos': Maximum number of downloads reached, stopping due to --max-downloads


Error al descargar videos: 'perros graciosos': Maximum number of downloads reached, stopping due to --max-downloads


Error al descargar videos: 'recetas faciles': Maximum number of downloads reached, stopping due to --max-downloads


He escogido yt-dlp en lugar de otras librerias ya que actualmente está mas actualizado  y se adapta mejor a los cambios que Youtube hace.
* Estructura:
    * Función descargar_videos(): Esta función descarga los videos de la lista de videos que se le pasa como argumento.
    * Función descargar_categorias(): Esta función recibe un diccionario con las categorias y sus respecticas consultas de busqueda. Itera sobre este diccionario y llama a la función descargar_videos() para descargar los videos de cada categoria. Ademas crea los directorios para cada categoria.
    * Si necesito modificar algo de la descarga lo hago en descargar_videos(). Si necesito añadir mas categorias, lo hago en descargar_categorias().

PARTE 2 - PROCESADOR DE VIDEOS

In [8]:
import cv2 # OpenCV es para abrir los videos, leer los fotogramas, cambiarles el tamaño y guardarlos como imagenes
import os # Esta libreria nos ayuda a crear carpetas y manejar las rutas de los archivos
import traceback # Esta libreria nos ayuda a imprimir errores detallados

def preprocesar_video(ruta_video, directorio_destino, framerate=3, resolucion=(224, 224)):
    """
    Procesa un video, extrayendo fotogramas y redimensionandolos.

    Args:
        ruta_video (str): Ruta al archivo de video.
        directorio_destino (str): Ruta al directorio de destino.
        framerate (int): Tasa de fotogramas.
        resolucion (tuple): Resolución de la imagen.
        """
    try:
        # 1. Obtener el nombre del video sin la extensión
        nombre_video = os.path.splitext(os.path.basename(ruta_video))[0]
        # 2. Crear el subdirectorio para el video
        directorio_video= os.path.join(directorio_destino, nombre_video)
        os.makedirs(directorio_video, exist_ok=True) # Crear el directorio si no existe, sin dar error si ya existe

        # 3. Abrir el video
        video = cv2.VideoCapture(ruta_video)
        # 4. Obtener el framerate original
        fps = video.get(cv2.CAP_PROP_FPS)
        if fps == 0: # Si el framerate es 0, se intenta obtener el framerate del video
            print (f"Error: No se pudo obtener el framerate del video {ruta_video}")
            fps = 30
        frame_count = 0 # Contador de frames guardados
        frame_rate_ratio = fps / framerate # Ratio entre el framerate original y el framerate deseado
        frame_number = 0 # Contador de frames totales

        # 5. Bucle para leer fotogramas
        while video.isOpened():
            ret, frame = video.read() # Leer el fotograma. ret indica si se ha leido el fotograma correctamente
            if not ret: # Si no se ha leido el fotograma correctamente, salir del bucle
                break

            # 6. Extraer fotograma segun el framerate deseado
            if frame_number % int(round(frame_rate_ratio)) == 0:
                # 7. Redimensionar el fotograma
                frame = cv2.resize(frame, resolucion)
                # 8. Guardar el fotograma
                nombre_archivo = os.path.join(directorio_video, f"{frame_count}.jpg")
                cv2.imwrite(nombre_archivo, frame)
                frame_count += 1
            frame_number += 1
        
        # 9. Liberar recursos
        video.release()
        print(f"Video {ruta_video} procesado exitosamente.{frame_count} fotogramas guardados en {directorio_video}.")
    except Exception as e: # Manejo de errores
        traceback.print_exc() # Imprimir el error detallado
        print(f"Error al procesar el video {ruta_video}: {e}")
    


In [ ]:

def procesar_videos(directorio_videos, directorio_destino, framerate=3, resolucion=(224, 224)):
    """
    Procesa todos los videos en un directorio.

    Args:
        directorio_videos (str): Ruta al directorio con los videos.
        directorio_destino (str): Ruta al directorio donde se guardarán los fotogramas.
        framerate (int): Tasa de fotogramas.
        resolucion (tuple): Resolución de la imagen.
    """
    for nombre_archivo in os.listdir(directorio_videos): # Iterar sobre los archivos en el directorio
        ruta_video = os.path.join(directorio_videos, nombre_archivo)    # Obtener la ruta del archivo
        if os.path.isfile(ruta_video): # Comprobar si es un archivo (video) y no un directorio
            preprocesar_video(ruta_video, directorio_destino, framerate, resolucion) # Procesar el video

# 10. Definir la carpeta principal donde se encuentran los videos
directorio_principal_videos = os.path.abspath("videos")

# 11. Definir las categorias
categorias = ["coches", "mascotas", "comida"]

# 12. Iterar sobre las categorias y procesar los videos
for categoria in categorias:
    # 13. Construir las rutas de entrada y salida
    directorio_videos_categoria = os.path.join(directorio_principal_videos, categoria) # Ruta de entrada a la carpeta videos
    directorio_destino_categoria = os.path.join("frames", categoria) # Ruta de salida a la carpeta frames
    
    # 14. Crear la carpeta destino si no existe
    os.makedirs(directorio_destino_categoria, exist_ok=True)
    
    # 15. Procesar el directorio de la categoria
    procesar_videos(directorio_videos_categoria, directorio_destino_categoria)


Video C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\videos\coches\$50 MILLION HYPERCAR GATHERING IN THE NETHERLANDS!-hQYRDNl-lGI.mp4 procesado exitosamente.2078 fotogramas guardados en frames\coches\$50 MILLION HYPERCAR GATHERING IN THE NETHERLANDS!-hQYRDNl-lGI.
Video C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\videos\coches\10 Coches deportivos míticos ＂Made in USA＂-3gx3LtEyzq8.mp4 procesado exitosamente.3613 fotogramas guardados en frames\coches\10 Coches deportivos míticos ＂Made in USA＂-3gx3LtEyzq8.
Video C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\videos\coches\10 coches deportivos muy recomendables： ¿nuevos o usados？-6IkT8obuk3o.mp4 procesado exitosamente.3341 fotogramas guardados en frames\coches\10 coches deportivos muy recomendables： ¿nuevos o usados？-6IkT8obuk3o.
Video C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\videos\coches\20 Coches Más Caros Del Mundo-SCuSEfVgAQg.mp4 procesado exitosamente.5871 fotogramas guardado

* EXPLICACIÓN: 
Tomamos videos que están ordenados por categorías en carpetas (ej: "videos/coches", "videos/mascotas", "videos/comida") y sacamos fotos (fotogramas) de esos videos. Estas fotos las guardamos en una carpeta nueva llamada "frames", manteniendo la misma organización por categorías (ej: "frames/coches", "frames/mascotas", "frames/comida"). Dentro de cada carpeta de categoría en "frames", creamos subcarpetas con el nombre del video original, y ahí dentro guardamos las fotos.